In [1]:
import sqlite3
import requests
import time
from bs4 import BeautifulSoup

In [2]:
# DBファイルの保存先パス（相対パスで指定）
path = ''
# DBファイル
db_name = 'google.db'
# DB接続の確立
# DBファイルが存在しない場合は新規作成される
conn = sqlite3.connect(db_name)
# DBへの接続を閉じる
cursor = conn.cursor()

In [3]:
path = ''
db_name = 'google.db'

try:
    # DB接続の確立
    conn = sqlite3.connect(path + db_name)

    # カーソルオブジェクトの作成
    cursor = conn.cursor()

    # SQL文の作成
    # テーブルの作成
    sql = 'CREATE TABLE googles(repo_path, language, stargazers_count);'

    # SQL文の実行
    cursor.execute(sql)

    # 変更をDBに反映
    conn.commit()

except sqlite3.Error as e:
    print('エラーが発生しました', e)

finally:
    # DBへの接続を閉じる
    conn.close()

エラーが発生しました table googles already exists


In [4]:
# --- 設定情報 ---
path = ''
db_name = 'google.db'
org = "google"
api_url = f"https://api.github.com/orgs/{org}/repos?per_page=100"

# GitHub API を使ってリポジトリ情報を取得
def fetch_github_repos(api_url):
    """
    GitHub API を使用して組織のリポジトリ一覧を取得する。
    """
    headers = {
        "Accept": "application/vnd.github+json",
        "User-Agent": "Mozilla/5.0"
    }

    repo_list = []
    page = 1

    while True:
        try:
            print(f"API取得中... page={page}")
            response = requests.get(f"{api_url}&page={page}", headers=headers)
            response.raise_for_status()
            repos = response.json()

            # データが空なら終了
            if not repos:
                break

            for repo in repos:
                repo_path = repo['full_name']             
                language = repo['language'] or "N/A"      
                stars = repo['stargazers_count']           

                repo_list.append((repo_path, language, stars))

            page += 1
            time.sleep(1)  # API負荷対策

        except requests.exceptions.RequestException as e:
            print(f"API取得中にエラー: {e}")
            break

    return repo_list

# main 処理（DBへ保存）
if __name__ == '__main__':
    conn = None
    try:
        conn = sqlite3.connect(path + db_name)
        cur = conn.cursor()

        # テーブル作成
        cur.execute("""
            CREATE TABLE IF NOT EXISTS google_repos (
                id INTEGER PRIMARY KEY,
                repo_path TEXT NOT NULL UNIQUE,
                language TEXT,
                stargazers_count INTEGER
            );
        """)
        conn.commit()

        # データ取得
        data = fetch_github_repos(api_url)

        if not data:
            print("取得データがありません。処理を終了します。")
            exit()

        # INSERT OR IGNORE で重複を無視
        sql = """
            INSERT OR IGNORE INTO google_repos 
            (repo_path, language, stargazers_count)
            VALUES (?, ?, ?);
        """

        cur.executemany(sql, data)
        conn.commit()

        print(f"完了: {len(data)}件のリポジトリ情報をDB({db_name})に保存しました。")

    except sqlite3.Error as e:
        print(f"DBエラー: {e}")

    finally:
        if conn:
            conn.close()

API取得中... page=1
API取得中... page=2
API取得中... page=3
API取得中... page=4
API取得中... page=5
API取得中... page=6
API取得中... page=7
API取得中... page=8
API取得中... page=9
API取得中... page=10
API取得中... page=11
API取得中... page=12
API取得中... page=13
API取得中... page=14
API取得中... page=15
API取得中... page=16
API取得中... page=17
API取得中... page=18
API取得中... page=19
API取得中... page=20
API取得中... page=21
API取得中... page=22
API取得中... page=23
API取得中... page=24
API取得中... page=25
API取得中... page=26
API取得中... page=27
API取得中... page=28
API取得中... page=29
API取得中... page=30
完了: 2812件のリポジトリ情報をDB(google.db)に保存しました。
